In [3]:
%%capture
pip install --upgrade firebase-admin 

In [4]:
import requests
from bs4 import BeautifulSoup
import datetime
import time
import threading
import queue
import random

In [5]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("../jsonFile/service-account.json")
    firebase_admin.initialize_app(cred)
    
db = firestore.client()

In [6]:
# docs = db.collection(u'cguscholar').document('faE3_ksAAAAJ').collection(u'updateTime').order_by(u'h_index.All', direction=firestore.Query.DESCENDING).limit(3).get()
# for doc in docs:
#     print(f'{doc.id} => {doc.to_dict()}')

In [7]:
docs = db.collection(u'cguscholar').order_by(u'updateTime').get()

for doc in docs:
    print(f'{doc.id}')

Ca8f9TEAAAAJ
NAzAaY0AAAAJ
QLxmlf0AAAAJ
6Z9AaHAAAAAJ
lLgQE20AAAAJ
0ZE58lcAAAAJ
IB3_fzoAAAAJ
ylK19VUAAAAJ
1ZO7pHkAAAAJ
bjEpXBoAAAAJ
faE3_ksAAAAJ
-8xAh5IAAAAJ
g-_ZXGsAAAAJ
OrFUuH0AAAAJ
Rfc8k-EAAAAJ
x_wsduUAAAAJ
CRo660MAAAAJ
DZ-fHPgAAAAJ
dE64QHAAAAAJ
lg39tc8AAAAJ
dYwbc9sAAAAJ
vGpPHEgAAAAJ
ojbLrDYAAAAJ
JG-n5IcAAAAJ
8ryT3pQAAAAJ
Z7OFTY0AAAAJ
k9qzaEcAAAAJ


In [8]:
def currentTime():
    now = datetime.datetime.now()
    currentTime = now.strftime("%Y-%m-%d %H:%M:%S")
    return currentTime

In [9]:
#personal detail
def personalData(soup):
    info = {}
    content = ''.join(soup.find('meta', property = 'og:image')['content']) #list to string

    d = soup.find('div', id = 'gsc_prf_i')

    #name
    info['name'] = d.find('div', id = 'gsc_prf_in').text

    #university
    try:
        info['university'] = d.find('a', class_ = 'gsc_prf_ila').text
    except:
        info['university'] = ' '

    #picture
    info['picture'] = soup.find('div', id = 'gsc_prf_pua').find('img')['src'] 
    
    label = []
    for p in soup.find_all('a', class_='gsc_prf_inta gs_ibl'):

        label.append(p.text)
    info['label'] = label
    info['updateTime'] = currentTime()
    return info


In [10]:
def citeBy(soup):
    citeBy = {}
    citations = {}
    h_index = {}
    i10_index = {}
    def cited(status, value):

        if status / 2 < 1:        
            if status % 2 == 0:
                citations['All'] = value
            else:
                citations['Since2016'] = value
            citeBy['citations'] = citations

        if status / 2 < 2:
            if status % 2 == 0:
                h_index['All'] = value
            else:
                h_index['Since2016'] = value
            citeBy['h_index'] = h_index

        if status / 2 < 3:
            if status % 2 == 0:
                i10_index['All'] = value
            else:
                i10_index['Since2016'] = value
            citeBy['i10_index'] = i10_index
            
    count_d = 0
    for d in soup.find_all('td', class_='gsc_rsb_std'):

        cited(count_d, d.text)
        count_d = count_d + 1       
    
    return citeBy

In [11]:
def result(soup, ID):
    infos = {}
    infos['id'] = ID
    infos['personalData'] = personalData(soup)
    infos['cited'] = citeBy(soup)

    return infos

In [12]:
def personalPage(id):
    url = 'https://scholar.google.com.tw/citations?hl=zh-TW&user=' + id
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    return result(soup, id)

In [13]:
import json
def jsontransfer(items):
    data = json.dumps(items)
    jsonStr = json.loads(data) 
    return jsonStr

In [14]:
def updatePersonal(userID):
    items = jsontransfer(personalPage(userID))
    print(items)
    ref = db.collection(u'cguscholar').document((items['id']))
    ref.collection(u'updateTime').document((items['personalData']['updateTime'])).set(items['cited'])
    ref.set(items['personalData'])

In [62]:
def IDCheck(ID):
    users_ref = db.collection(u'cguscholar').document(ID)
    doc = users_ref.get()
    if doc.exists:
        checkTemp = doc.to_dict()
        Timestamp = checkTemp['updateTime']
        return Timestamp
    else:
        return ('Not found')

In [63]:
def expiresCheck(last_update, expires):
    if last_update == 'Not found':
        compare = True
    else:
        expires_date = datetime.datetime.strptime(last_update, "%Y-%m-%d %H:%M:%S")
        compare_date = expires_date + datetime.timedelta(days = expires)
        current_date = datetime.datetime.now()

        compare = compare_date < current_date
    print(compare)
    return compare # compare result
    #過期或Not found為true

In [64]:
def ID_queue(label):
    # 建立佇列
    ID_queue = queue.Queue()

    # 將資料放入佇列
    label_ref = db.collection(u'Label-Domain').document(label)
    docs = label_ref.get()
    IDtemp = docs.to_dict()
    
    #取五個過期或為爬過的ID
    number = 5 
    ID_count = 0
    while (number != 0):        
        expire_time = IDCheck(IDtemp['userID'][ID_count])
        if(expiresCheck(expire_time, 1)):
            print(IDtemp['userID'][ID_count])
            ID_queue.put(IDtemp['userID'][ID_count])
            number = number - 1
        ID_count = ID_count + 1
        
    return ID_queue

In [65]:
def userDataList(label):
    
    # Worker 類別，負責處理資料
    class Worker(threading.Thread):
      def __init__(self, queue, num):
        threading.Thread.__init__(self)
        self.queue = queue
        self.num = num

      def run(self):
        while self.queue.qsize() > 0:
          # 取得新的資料
          userID = self.queue.get()

          # 處理資料
          #print("Worker %d: %s" % (self.num, userID))
          updatePersonal(userID)

          time.sleep(1)
    work_queue = ID_queue(label)
    # 建立兩個 Worker
    my_worker1 = Worker(work_queue, 1)
    my_worker2 = Worker(work_queue, 2)

    # 讓 Worker 開始處理資料
    my_worker1.start()
    my_worker2.start()

    # 等待所有 Worker 結束
    my_worker1.join()
    my_worker2.join()

    print("Done.")
    


In [66]:
label = 'causal_inference'
userDataList(label)

False
False
False
False
False
False
False
False
False
False
True
L71WymMAAAAJ
True
3r-fWJwAAAAJ
True
2tt6ZJ0AAAAJ
True
8zZofa8AAAAJ
True
6F3ZIeEAAAAJ
{'id': 'L71WymMAAAAJ', 'personalData': {'name': 'Nick Graetz', 'university': '首頁', 'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=L71WymMAAAAJ&citpid=1', 'label': ['Structural racism', 'housing', 'causal inference', 'social policy', 'political economy'], 'updateTime': '2021-09-28 15:18:52'}, 'cited': {'citations': {'All': '39625', 'Since2016': '37083'}, 'h_index': {'All': '25', 'Since2016': '25'}, 'i10_index': {'All': '30', 'Since2016': '30'}}}
{'id': '3r-fWJwAAAAJ', 'personalData': {'name': 'Peter Bühlmann', 'university': 'ETH Zurich', 'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=3r-fWJwAAAAJ&citpid=5', 'label': ['Statistics', 'Machine Learning', 'Causal Inference', 'Computational Biology'], 'updateTime': '2021-09-28 15:18:52'}, 'cited': {'citations': {'All': '38

In [88]:
# jsonFile = open("jsonFile/personalData.json", "w")
# jsonFile.write(jsonStr)
# jsonFile.close()